목적 : unlisted data Modeling from standard scaled  
===================

import modules

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from xgboost import XGBClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

In [9]:
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: c:\Users\LG\2ndProject\비상장\3_모델링


read csv files

In [10]:
train = pd.read_csv('../datasets/Train_Test/unlisted_resampled_train_data.csv')
test = pd.read_csv('../datasets/Train_Test/unlisted_test_data.csv')

In [11]:
# train_life_cycle = train[['도입기', '성장기', '성숙기', '쇠퇴기']]
# test_life_cycle = test[['도입기', '성장기', '성숙기', '쇠퇴기']]

selected features

In [12]:
selected_features = ['CASH FLOW 대 부채비율', 'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '차입금의존도', '순운전자본비율',
                     '자기자본구성비율', '경영자본순이익률', '총자본사업이익률', '총자본영업이익률', '금융비용부담률',
                     '매출액증가율', '이윤분배율', '총자본회전률', '영업년수',
                     '도입기', '성장기', '성숙기', '쇠퇴기']

x_train = train[selected_features]
x_test = test[selected_features]

y_train = train['부실판단']
y_test = test['부실판단']

test 결측치 처리

In [13]:
x_test = x_test.fillna(test['영업년수'].median())

# describe

In [14]:
x_train[train['부실판단'] == 0].describe()

,CASH FLOW 대 부채비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,차입금의존도,순운전자본비율,자기자본구성비율,경영자본순이익률,총자본사업이익률,총자본영업이익률,금융비용부담률,매출액증가율,이윤분배율,총자본회전률,영업년수,도입기,성장기,성숙기,쇠퇴기
count,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000
mean,11.509313,2.554777,2.178420,37.452623,9.015978,42.426205,3.176920,3.514433,3.113734,4.456140,6.112282,-6.879044,1.119652,20.884139,0.301790,0.163683,0.210184,0.324343
std,54.778702,13.554067,83.813895,24.879230,33.420187,24.253391,12.207293,8.992482,8.976275,15.691608,54.648609,147.166229,1.075157,10.256470,0.459088,0.370031,0.407486,0.468183
min,-165.260800,-42.490200,-572.428800,0.000000,-76.020400,0.720000,-49.339200,-32.002600,-32.627200,-47.190000,-97.543000,-1437.891400,0.010000,4.000000,0.000000,0.000000,0.000000,0.000000
25%,-4.330000,-2.610000,-2.950000,15.810000,-12.270000,23.950000,0.010000,0.310000,-0.100000,0.070000,-16.270000,0.090000,0.410000,13.000000,0.000000,0.000000,0.000000,0.000000
50%,2.390000,1.450000,1.490000,38.370000,8.340000,38.390000,2.170000,3.280000,2.940000,0.940000,2.820000,16.930000,0.850000,20.000000,0.000000,0.000000,0.000000,0.000000
75%,12.820000,6.310000,8.430000,56.200000,30.830000,60.080000,6.910000,7.150000,6.750000,2.910000,19.970000,38.320000,1.450000,27.000000,1.000000,0.000000,0.000000,1.000000
max,788.472000,145.442000,743.573800,92.597500,90.996800,97.423800,74.888400,50.366000,49.890000,142.110800,432.832000,86.144400,7.252600,55.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
x_train[train['부실판단'] == 1].describe()

,CASH FLOW 대 부채비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,차입금의존도,순운전자본비율,자기자본구성비율,경영자본순이익률,총자본사업이익률,총자본영업이익률,금융비용부담률,매출액증가율,이윤분배율,총자본회전률,영업년수,도입기,성장기,성숙기,쇠퇴기
count,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000,4301.000000
mean,-8.229970,-3.599775,-51.925289,51.853057,-9.566183,28.325831,-4.522224,-2.805103,-3.201570,14.313713,43.487154,-92.710246,0.703075,22.110346,0.301790,0.163683,0.210184,0.324343
std,34.006612,11.421473,240.279797,28.134917,38.315165,24.741688,16.090219,9.669048,9.628542,40.224729,220.435705,352.812458,0.795058,11.846056,0.459088,0.370031,0.407486,0.468183
min,-348.859200,-66.805600,-2174.418800,0.000000,-121.618400,0.000000,-96.701400,-60.590300,-60.681000,-138.847200,-91.373200,-4063.945200,0.010000,4.000000,0.000000,0.000000,0.000000,0.000000
25%,-10.590000,-6.870000,-19.280000,32.440000,-33.220000,7.990000,-7.440000,-4.950000,-5.210000,0.850000,-16.670000,-56.950000,0.110000,13.000000,0.000000,0.000000,0.000000,0.000000
50%,-2.440000,-1.720000,-3.400000,53.990000,-7.810000,23.060000,-1.810000,-0.590000,-0.930000,2.940000,0.070000,0.000000,0.470000,20.000000,0.000000,0.000000,0.000000,0.000000
75%,2.250000,1.570000,3.630000,71.500000,13.530000,41.910000,0.880000,1.710000,1.300000,10.990000,21.130000,8.480000,0.960000,28.000000,1.000000,0.000000,0.000000,1.000000
max,119.656000,54.232400,315.224300,124.008000,89.081700,97.928000,75.338400,21.816000,21.375600,350.081900,2135.799400,91.481600,5.049800,62.970000,1.000000,1.000000,1.000000,1.000000


# standard scaling

In [7]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

labels = x_train.columns

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

x_train = pd.DataFrame(data=x_train, columns = labels)
x_test = pd.DataFrame(data=x_test, columns = labels)

# modeling

In [11]:
validation = pd.DataFrame()
default = pd.DataFrame()
best = pd.DataFrame()

## LogisticRegression

기본 모델

In [12]:
# 모델 생성
logit_model = LogisticRegression()

# 모델 학습 및 평가
logit_model.fit(x_train, y_train)
y_pred = logit_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.3f}')
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 스코어: {f1:.3f}')
print(f'ROC AUC 스코어: {roc_auc:.3f}')

# 결과 저장
result = {
    'Accuracy' : round(accuracy, 3),
    'Precision' : round(precision, 3),
    'Recall' : round(recall, 3),
    'F1' : round(f1, 3),
    'ROC AUC' : round(roc_auc, 3)
}
default['LogisticRegression'] = result

Accuracy: 0.731
Precision: 0.617
Recall: 0.712
F1 스코어: 0.661
ROC AUC 스코어: 0.727


파라미터 조정

In [13]:
# 하이퍼파라미터 범위 지정
param_grid = {
    'C': np.arange(0, 100.0, 0.01),  # 규제 강도
    'penalty': ['l1', 'l2', 'elasticnet'],  # 규제 유형
    'solver': ['liblinear', 'saga']  # 최적화 알고리즘
}

model = LogisticRegression()
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=99,
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_search.fit(x_train, y_train)

best_model = random_search.best_estimator_

y_pred = best_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("Logistic 하이퍼파라미터 조정 평가 지표")
print("Best Parameters:", random_search.best_params_)
print(f'Accuracy: {accuracy:.3f}')
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')
print(f'ROC AUC Score: {roc_auc:.3f}')

# 파라미터 조절 결과
result0_logit = {
    'Accuracy' : round(accuracy, 3),
    'Precision' : round(precision, 3),
    'Recall' : round(recall, 3),
    'ROC AUC' : round(roc_auc, 3),
    'Best Parameters' : random_search.best_params_
}
best['LogisticRegression'] = result0_logit


Fitting 5 folds for each of 99 candidates, totalling 495 fits
Logistic 하이퍼파라미터 조정 평가 지표
Best Parameters: {'solver': 'liblinear', 'penalty': 'l1', 'C': 10.66}
Accuracy: 0.731
Precision: 0.617
Recall: 0.710
F1 Score: 0.660
ROC AUC Score: 0.727


최적 파라미터 검증

In [18]:
# 모델 생성
cv_logit_model = LogisticRegression(random_search.best_params_)

# Cross Validation
cv_accuracy = cross_val_score(cv_logit_model, x_train, y_train, cv=7, scoring='accuracy')
cv_precision = cross_val_score(cv_logit_model, x_train, y_train, cv=7, scoring='precision')
cv_recall = cross_val_score(cv_logit_model, x_train, y_train, cv=7, scoring='recall')
cv_f1 = cross_val_score(cv_logit_model, x_train, y_train, cv=7, scoring='f1')
cv_roc_auc = cross_val_score(cv_logit_model, x_train, y_train, cv=7, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_logit = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['LogisticRegression'] = cv_result_logit

CV_Accuracy_Scores: [0.69812856 0.70545159 0.79332791 0.79251424 0.77868186 0.67534581
 0.66693811]
CV_Precision_Scores: [0.70628183 0.725      0.77950311 0.76086957 0.77179081 0.67704918
 0.67950963]
CV_Recall_Scores: [0.67752443 0.66123779 0.81758958 0.85365854 0.79186992 0.67154472
 0.63192182]
CV_F1_Scores: [0.69160432 0.69165247 0.79809221 0.8045977  0.78170144 0.67428571
 0.65485232]
CV_ROC/AUC: [0.75658484 0.78024417 0.87333492 0.87555944 0.85711713 0.7465851
 0.74730236]

=======교차검증 결과=======
CV_Accuracy_mean: 0.730
CV_Precision_mean: 0.729
CV_Recall_mean: 0.729
CV_F1_스코어_mean: 0.728
CV_ROC_AUC+스코어_mean: 0.805


## RandomForestClassifier

기본 모델

In [19]:
# Random Forest 모델 생성 및 학습
rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_rf = rf_model.predict(x_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_pred_rf)

print("Random Forest 모델 평가 지표")
print(f'Accuracy: {accuracy_rf:.3f}')
print(f'Precision: {precision_rf:.3f}')
print(f'Recall: {recall_rf:.3f}')
print(f'F1 스코어: {f1_rf:.3f}')
print(f'ROC AUC 스코어: {roc_auc_rf:.3f}')

# 기본결과 저장
result_rf = {
    'Accuracy' : round(accuracy_rf, 3),
    'Precision' : round(precision_rf, 3),
    'Recall' : round(recall_rf, 3),
    'F1' : round(f1_rf, 3),
    'ROC AUC' : round(roc_auc_rf, 3)
}
default['RandomForestClassifier'] = result_rf
default


Random Forest 모델 평가 지표
Accuracy: 0.807
Precision: 0.710
Recall: 0.804
F1 스코어: 0.754
ROC AUC 스코어: 0.806


파라미터 조정

In [21]:
random_search = {'n_estimators': np.arange(10, 200, 10),
                 'max_depth': np.arange(1, 10, 1),
                 'min_samples_split': np.arange(1, 9, 1),
                 'min_samples_leaf': np.arange(1, 9, 1)}

clf = RandomForestClassifier()
random_rf = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 99,
                               cv = 7, verbose= 1, random_state= 101, n_jobs = -1)
random_rf.fit(x_train,y_train)
random_pf = random_rf.best_estimator_.predict(x_test)

accuracy_rdrf = accuracy_score(y_test, random_pf)
precision_rdrf = precision_score(y_test, random_pf)
recall_rdrf = recall_score(y_test, random_pf)
f1_rdrf = f1_score(y_test, random_pf)
roc_auc_rdrf = roc_auc_score(y_test, random_pf)

print("Random Forest 모델 하이퍼파라미터 조정 평가 지표")
print(random_rf.best_params_)
print(f'Accuracy: {accuracy_rdrf:.3f}')
print(f'Precision: {precision_rdrf:.3f}')
print(f'Recall: {recall_rdrf:.3f}')
print(f'F1 스코어: {f1_rdrf:.3f}')
print(f'ROC AUC 스코어: {roc_auc_rdrf:.3f}')

# 파라미터 조정 결과 저장
result0_rf = {
    'Accuracy' : round(accuracy_rdrf, 3),
    'Precision' : round(precision_rdrf, 3),
    'Recall' : round(recall_rdrf, 3),
    'F1' : round(f1_rdrf, 3),
    'ROC AUC' : round(roc_auc_rdrf, 3),
    'Best Parameters' : random_rf.best_params_
}
best['RandomForestClassifier'] = result0_rf
best

Fitting 7 folds for each of 99 candidates, totalling 693 fits
Random Forest 모델 하이퍼파라미터 조정 평가 지표
{'n_estimators': 70, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_depth': 9}
Accuracy: 0.810
Precision: 0.700
Recall: 0.846
F1 스코어: 0.766
ROC AUC 스코어: 0.818


최적 파라미터 검증

In [24]:
# 모델 생성
cv_model_rf = RandomForestClassifier(n_estimators=70, min_samples_split=3, min_samples_leaf=1, max_depth=9)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_rf, x_train, y_train, cv=7, scoring='accuracy')
cv_precision = cross_val_score(cv_model_rf, x_train, y_train, cv=7, scoring='precision')
cv_recall = cross_val_score(cv_model_rf, x_train, y_train, cv=7, scoring='recall')
cv_f1 = cross_val_score(cv_model_rf, x_train, y_train, cv=7, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_rf, x_train, y_train, cv=7, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_rf = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['RandomForestClassifier'] = cv_result_rf

CV_Accuracy_Scores: [0.73311635 0.75264443 0.86004882 0.90398698 0.84865745 0.7103336
 0.71579805]
CV_Precision_Scores: [0.69500675 0.69708995 0.82937685 0.88117284 0.84493671 0.67241379
 0.68306011]
CV_Recall_Scores: [0.84201954 0.8485342  0.90390879 0.93333333 0.86504065 0.81626016
 0.79641694]
CV_F1_Scores: [0.76091784 0.77239354 0.86091686 0.90143084 0.84892086 0.74340176
 0.73082707]
CV_ROC/AUC: [0.81589471 0.83843913 0.92473451 0.95887026 0.93437144 0.7938508
 0.79345669]

=======교차검증 결과=======
CV_Accuracy_mean: 0.789
CV_Precision_mean: 0.758
CV_Recall_mean: 0.858
CV_F1_스코어_mean: 0.803
CV_ROC_AUC+스코어_mean: 0.866


## AdaBoostClassifier

기본 모델

In [25]:
# AdaBoost 모델 생성 및 학습
adaboost_model = AdaBoostClassifier()
adaboost_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_adaboost = adaboost_model.predict(x_test)

accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
precision_adaboost = precision_score(y_test, y_pred_adaboost)
recall_adaboost = recall_score(y_test, y_pred_adaboost)
f1_adaboost = f1_score(y_test, y_pred_adaboost)
roc_auc_adaboost = roc_auc_score(y_test, y_pred_adaboost)

print("AdaBoost 모델 평가 지표:")
print(f'Accuracy: {accuracy_adaboost:.3f}')
print(f'Precision: {precision_adaboost:.3f}')
print(f'Recall: {recall_adaboost:.3f}')
print(f'F1 스코어: {f1_adaboost:.3f}')
print(f'ROC AUC 스코어: {roc_auc_adaboost:.3f}')

AdaBoost 모델 평가 지표:
Accuracy: 0.776
Precision: 0.678
Recall: 0.744
F1 스코어: 0.710
ROC AUC 스코어: 0.769


In [27]:
# 기본 결과
result = {
    'Accuracy' : round(accuracy_adaboost, 3),
    'Precision' : round(precision_adaboost, 3),
    'Recall' : round(recall_adaboost, 3),
    'F1' : round(f1_adaboost, 3),
    'ROC AUC' : round(roc_auc_adaboost, 3)
}
default['AdaBoostClassifier'] = result
default

,LogisticRegression,RandomForestClassifier,AdaBoostClassifier
Accuracy,0.731,0.807,0.776
Precision,0.617,0.710,0.678
Recall,0.712,0.804,0.744
F1,0.661,0.754,0.710
ROC AUC,0.727,0.806,0.769


파라미터 조정

In [28]:
# randon search AdaBoostClassifier
adaboost_model = AdaBoostClassifier()

param_grid = {
    'n_estimators': np.arange(10, 300, 10),  # 트리 개수
    'learning_rate': np.arange(0.01, 1, 0.01)  # 학습률
}

random_search = RandomizedSearchCV(estimator=adaboost_model, param_distributions=param_grid, n_iter=99,
                                   scoring='accuracy', cv=7, verbose=1, random_state=42)

random_search.fit(x_train, y_train)

best_model = random_search.best_estimator_

y_pred_adaboost = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
precision_adaboost = precision_score(y_test, y_pred_adaboost)
recall_adaboost = recall_score(y_test, y_pred_adaboost)
f1_adaboost = f1_score(y_test, y_pred_adaboost)
roc_auc_adaboost = roc_auc_score(y_test, y_pred_adaboost)

print("Best Parameters:", random_search.best_params_)
print("AdaBoost 모델 하이퍼파라미터 조정 평가 지표:")
print(f'Accuracy: {accuracy_adaboost:.3f}')
print(f'Precision: {precision_adaboost:.3f}')
print(f'Recall: {recall_adaboost:.3f}')
print(f'F1 Score: {f1_adaboost:.3f}')
print(f'ROC AUC Score: {roc_auc_adaboost:.3f}')

# 파라미터 조정 결과 저장
result0_adaboost = {
    'Accuracy' : round(accuracy_adaboost, 3),
    'Precision' : round(precision_adaboost, 3),
    'Recall' : round(recall_adaboost, 3),
    'F1' : round(f1_adaboost, 3),
    'ROC AUC' : round(roc_auc_adaboost, 3),
    'Best Parameters' : random_search.best_params_
}
best['AdaBoostClassifier'] = result0_adaboost
best

Fitting 7 folds for each of 99 candidates, totalling 693 fits
Best Parameters: {'n_estimators': 250, 'learning_rate': 0.78}
AdaBoost 모델 하이퍼파라미터 조정 평가 지표:
Accuracy: 0.779
Precision: 0.680
Recall: 0.756
F1 Score: 0.716
ROC AUC Score: 0.774


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier
Accuracy,0.731,0.81,0.779
Precision,0.617,0.7,0.68
Recall,0.712,0.846,0.756
ROC AUC,0.727,0.818,0.774
Best Parameters,"{'solver': 'saga', 'penalty': 'l2', 'C': 4.9}","{'n_estimators': 70, 'min_samples_split': 3, '...","{'n_estimators': 250, 'learning_rate': 0.78}"


최적 파라미터 검증

In [30]:
# 모델 생성
# AdaBoostClassifier 최적파라미터 적용 Crossvalidation
cv_model_adaboost = AdaBoostClassifier(n_estimators=250, learning_rate=0.78)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_adaboost, x_train, y_train, cv=7, scoring='accuracy')
cv_precision = cross_val_score(cv_model_adaboost, x_train, y_train, cv=7, scoring='precision')
cv_recall = cross_val_score(cv_model_adaboost, x_train, y_train, cv=7, scoring='recall')
cv_f1 = cross_val_score(cv_model_adaboost, x_train, y_train, cv=7, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_adaboost, x_train, y_train, cv=7, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_adaboost = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['AdaBoostClassifier'] = cv_result_adaboost

CV_Accuracy_Scores: [0.73555736 0.75183076 0.8096013  0.87713588 0.84214809 0.72416599
 0.73371336]
CV_Precision_Scores: [0.73649755 0.74484945 0.83807829 0.88157895 0.85860307 0.715625
 0.7310789 ]
CV_Recall_Scores: [0.73289902 0.76547231 0.76710098 0.87154472 0.8195122  0.74471545
 0.73941368]
CV_F1_Scores: [0.73469388 0.75502008 0.80102041 0.87653312 0.83860233 0.72988048
 0.73522267]
CV_ROC/AUC: [0.83034348 0.84379121 0.9018776  0.95023437 0.91879188 0.82419295
 0.81860285]

=======교차검증 결과=======
CV_Accuracy_mean: 0.782
CV_Precision_mean: 0.787
CV_Recall_mean: 0.777
CV_F1_스코어_mean: 0.782
CV_ROC_AUC+스코어_mean: 0.870


## BaggingClassifier

기본 모델

In [31]:
# Bagging 모델 생성 및 학습
bagging_model = BaggingClassifier()
bagging_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_bagging = bagging_model.predict(x_test)

accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
precision_bagging = precision_score(y_test, y_pred_bagging)
recall_bagging = recall_score(y_test, y_pred_bagging)
f1_bagging = f1_score(y_test, y_pred_bagging)
roc_auc_bagging = roc_auc_score(y_test, y_pred_bagging)

print("Bagging 모델 평가 지표:")
print(f'Accuracy: {accuracy_bagging:.3f}')
print(f'Precision: {precision_bagging:.3f}')
print(f'Recall: {recall_bagging:.3f}')
print(f'F1 스코어: {f1_bagging:.3f}')
print(f'ROC AUC 스코어: {roc_auc_bagging:.3f}')

# 결과 저장
# 기본값 저장
result_bagging = {
    'Accuracy' : round(accuracy_bagging, 3),
    'Precision' : round(precision_bagging, 3),
    'Recall' : round(recall_bagging, 3),
    'F1' : round(f1_bagging, 3),
    'ROC AUC' : round(roc_auc_bagging, 3)
}
default['BaggingClassifier'] = result_bagging
default

Bagging 모델 평가 지표:
Accuracy: 0.794
Precision: 0.714
Recall: 0.734
F1 스코어: 0.724
ROC AUC 스코어: 0.781


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier
Accuracy,0.731,0.807,0.776,0.794
Precision,0.617,0.710,0.678,0.714
Recall,0.712,0.804,0.744,0.734
F1,0.661,0.754,0.710,0.724
ROC AUC,0.727,0.806,0.769,0.781


파라미터 조정

In [ ]:
# random search
bagging_model = BaggingClassifier()

param_grid = {
    'n_estimators': np.arange(10, 100, 10),
    'max_samples': np.arange(0.1, 1.0, 0.1),
    'max_features': np.arange(0.1, 1, 0.1)
}

random_search = RandomizedSearchCV(estimator=bagging_model, param_distributions=param_grid, n_iter=99,
                                   scoring='accuracy', cv=7, verbose=1, random_state=42)

random_search.fit(x_train, y_train)

# 최적의 모델 저장
best_model = random_search.best_estimator_

# 최적의 모델로 예측 수행
y_pred_bagging = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
precision_bagging = precision_score(y_test, y_pred_bagging)
recall_bagging = recall_score(y_test, y_pred_bagging)
f1_bagging = f1_score(y_test, y_pred_bagging)
roc_auc_bagging = roc_auc_score(y_test, y_pred_bagging)

print("Best Parameters:", random_search.best_params_)
print("Bagging 모델 하이퍼파라미터 평가 지표:")
print(f'Accuracy: {accuracy_bagging:.3f}')
print(f'Precision: {precision_bagging:.3f}')
print(f'Recall: {recall_bagging:.3f}')
print(f'F1 Score: {f1_bagging:.3f}')
print(f'ROC AUC Score: {roc_auc_bagging:.3f}')

# 파라미터 조정 결과 저장
result0_bagging = {
    'Accuracy' : round(accuracy_bagging, 3),
    'Precision' : round(precision_bagging, 3),
    'Recall' : round(recall_bagging, 3),
    'F1' : round(f1_bagging, 3),
    'ROC AUC' : round(roc_auc_bagging, 3),
    'Best Parameters' : random_search.best_params_
}
best['BaggingClassifier'] = result0_bagging
best

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Parameters: {'n_estimators': 50, 'max_samples': 0.7, 'max_features': 0.7}
Bagging 모델 하이퍼파라미터 평가 지표:
Accuracy: 0.802
Precision: 0.798
Recall: 0.822
F1 Score: 0.810
ROC AUC Score: 0.802


최적 파라미터 검증

In [32]:
# 모델 생성
# BaggingClassifier 최적파라미터 적용 Crossvalidation
cv_model_bagging = BaggingClassifier(n_estimators=50, max_samples=0.7, max_features=0.7)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_bagging, x_train, y_train, cv=7, scoring='accuracy')
cv_precision = cross_val_score(cv_model_bagging, x_train, y_train, cv=7, scoring='precision')
cv_recall = cross_val_score(cv_model_bagging, x_train, y_train, cv=7, scoring='recall')
cv_f1 = cross_val_score(cv_model_bagging, x_train, y_train, cv=7, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_bagging, x_train, y_train, cv=7, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_bagging = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['BaggingClassifier'] = cv_result_bagging

CV_Accuracy_Scores: [0.73637103 0.73230269 0.84865745 0.90317331 0.85353946 0.70463792
 0.70439739]
CV_Precision_Scores: [0.7029703  0.7105644  0.83001531 0.89064976 0.84185304 0.66997167
 0.70245399]
CV_Recall_Scores: [0.78013029 0.78013029 0.86156352 0.91707317 0.84552846 0.78373984
 0.72801303]
CV_F1_Scores: [0.74767802 0.75646045 0.85419968 0.8983871  0.84715447 0.73684211
 0.71790808]
CV_ROC/AUC: [0.82195387 0.82932655 0.92254575 0.95869416 0.93121607 0.79634808
 0.78130139]

=======교차검증 결과=======
CV_Accuracy_mean: 0.783
CV_Precision_mean: 0.764
CV_Recall_mean: 0.814
CV_F1_스코어_mean: 0.794
CV_ROC_AUC+스코어_mean: 0.863


## SVC_linear

SVC linear 기본모델

In [ ]:
# SVM 모델 생성 및 학습
svm_model = SVC(kernel='linear')
svm_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_svm = svm_model.predict(x_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

print("SVM 모델 평가 지표:")
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 스코어: {f1_svm:.3f}')
print(f'ROC AUC 스코어: {roc_auc_svm:.3f}')

# 결과 저장
result_linear = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3)
}
{}
default['SVC_linear'] = result_linear
default

SVM 모델 평가 지표:
Accuracy: 0.802
Precision: 0.798
Recall: 0.822
F1 스코어: 0.810
ROC AUC 스코어: 0.782


SVC linear 파라미터 조정

In [33]:
svm_model = SVC()

param_grid = {
    'C': [0.1, 1, 10, 100],  # 규제 매개변수
    'gamma': [0.1, 0.01, 0.001, 0.0001],  # 커널 계수
    'kernel': ['linear']  # 커널 타입
}

random_search = RandomizedSearchCV(estimator=svm_model, param_distributions=param_grid, n_iter=99,
                                   scoring='accuracy', cv=7, verbose=1, random_state=42)

random_search.fit(x_train, y_train)

print("Best Parameters:", random_search.best_params_)
best_model = random_search.best_estimator_

y_pred_svm = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

# 평가 메트릭 출력
print("SVM 모델 평가 지표:")
print("Best Parameters:", random_search.best_params_)
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 Score: {f1_svm:.3f}')
print(f'ROC AUC Score: {roc_auc_svm:.3f}')

# 파라미터 조정 결과 저장
result0_svc_linear = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3),
    'Best Parameters' : random_search.best_params_
}
best['SVC_linear'] = result0_svc_linear
best

Fitting 7 folds for each of 16 candidates, totalling 112 fits
Best Parameters: {'kernel': 'linear', 'gamma': 0.1, 'C': 10}
SVM 모델 평가 지표:
Best Parameters: {'kernel': 'linear', 'gamma': 0.1, 'C': 10}
Accuracy: 0.729
Precision: 0.613
Recall: 0.715
F1 Score: 0.660
ROC AUC Score: 0.726


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,SVC_linear
Accuracy,0.731,0.81,0.779,0.729
Precision,0.617,0.7,0.68,0.613
Recall,0.712,0.846,0.756,0.715
ROC AUC,0.727,0.818,0.774,0.726
Best Parameters,"{'solver': 'saga', 'penalty': 'l2', 'C': 4.9}","{'n_estimators': 70, 'min_samples_split': 3, '...","{'n_estimators': 250, 'learning_rate': 0.78}","{'kernel': 'linear', 'gamma': 0.1, 'C': 10}"


SVC linear 최적 파라미터 검증

In [35]:
# 모델 생성
# SVC_linear 최적파라미터 적용 Crossvalidation
cv_model_scv_linear = SVC(kernel='linear', gamma=0.1, C=10)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=7, scoring='accuracy')
cv_precision = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=7, scoring='precision')
cv_recall = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=7, scoring='recall')
cv_f1 = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=7, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=7, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_linear = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['SVC_linear'] = cv_result_linear
validation

CV_Accuracy_Scores: [0.69650122 0.70870627 0.78844589 0.79332791 0.79007323 0.67778682
 0.66938111]
CV_Precision_Scores: [0.71103327 0.72775801 0.77147239 0.75822604 0.78110236 0.6752
 0.68181818]
CV_Recall_Scores: [0.66123779 0.66612378 0.81921824 0.86178862 0.80650407 0.68617886
 0.63517915]
CV_F1_Scores: [0.68523207 0.69557823 0.79462875 0.80669711 0.7936     0.68064516
 0.65767285]
CV_ROC/AUC: [0.75479463 0.77970128 0.87467228 0.87987871 0.86207463 0.74465719
 0.74591773]

=======교차검증 결과=======
CV_Accuracy_mean: 0.732
CV_Precision_mean: 0.730
CV_Recall_mean: 0.734
CV_F1_스코어_mean: 0.731
CV_ROC_AUC+스코어_mean: 0.806


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear
Accuracy,0.730,0.789,0.782,0.783,0.732
Precision,0.729,0.758,0.787,0.764,0.730
Recall,0.729,0.858,0.777,0.814,0.734
F1,0.728,0.803,0.782,0.794,0.731
ROC AUC,0.805,0.866,0.870,0.863,0.806


## SVC_RBF

In [36]:
# SVM 모델 생성 및 학습
svm_model = SVC(kernel='rbf')
svm_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_svm = svm_model.predict(x_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

print("SVM 모델 평가 지표:")
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 스코어: {f1_svm:.3f}')
print(f'ROC AUC 스코어: {roc_auc_svm:.3f}')

result_svc_rbf = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3)
}
default['SVC_rbf'] = result_svc_rbf
default

SVM 모델 평가 지표:
Accuracy: 0.772
Precision: 0.662
Recall: 0.779
F1 스코어: 0.716
ROC AUC 스코어: 0.774


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_rbf
Accuracy,0.731,0.807,0.776,0.794,0.772
Precision,0.617,0.710,0.678,0.714,0.662
Recall,0.712,0.804,0.744,0.734,0.779
F1,0.661,0.754,0.710,0.724,0.716
ROC AUC,0.727,0.806,0.769,0.781,0.774


파라미터 조정

In [15]:
svm_model = SVC()

param_grid = {
    'C': np.arange(0.0001, 10, 0.0001),  # Adjust the maximum value
    'gamma': np.arange(0.0001, 1, 0.0001),  # Keep the range as is
    'kernel': ['rbf']
}

random_search = RandomizedSearchCV(estimator=svm_model, param_distributions=param_grid, n_iter=99,
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_search.fit(x_train, y_train)

print("Best Parameters:", random_search.best_params_)
best_model = random_search.best_estimator_

y_pred_svm = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

# 평가 메트릭 출력
print("SVM 모델 평가 지표:")
print("Best Parameters:", random_search.best_params_)
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 Score: {f1_svm:.3f}')
print(f'ROC AUC Score: {roc_auc_svm:.3f}')

# 파라미터 조정 결과 저장
result0_svc_rbf = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3),
    'Best Parameters' : random_search.best_params_
}
best['SVC_rbf'] = result0_svc_rbf
best

Fitting 5 folds for each of 99 candidates, totalling 495 fits


Best Parameters: {'kernel': 'rbf', 'gamma': 0.0597, 'C': 3.8920000000000003}
SVM 모델 평가 지표:
Best Parameters: {'kernel': 'rbf', 'gamma': 0.0597, 'C': 3.8920000000000003}
Accuracy: 0.782
Precision: 0.675
Recall: 0.783
F1 Score: 0.725
ROC AUC Score: 0.782


,LogisticRegression,SVC_rbf
Accuracy,0.731,0.782
Precision,0.617,0.675
Recall,0.71,0.783
ROC AUC,0.727,0.782
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l1', 'C': ...","{'kernel': 'rbf', 'gamma': 0.0597, 'C': 3.8920..."


svc rbf 최적 파라미터 검증

In [17]:
# 모델 생성
# SVC_linear 최적파라미터 적용 Crossvalidation
cv_model_scv_rbf = SVC(kernel='rbf', gamma=0.0597, C=3.892)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_rbf = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['SVC_rbf'] = cv_result_rbf

CV_Accuracy_Scores: [0.73910517 0.79256246 0.88604651 0.775      0.70232558]
CV_Precision_Scores: [0.74377224 0.78701595 0.85169492 0.76424581 0.70567376]
CV_Recall_Scores: [0.72906977 0.80255517 0.93488372 0.79534884 0.69418605]
CV_F1_Scores: [0.73634762 0.7947096  0.89135255 0.77948718 0.69988277]
CV_ROC/AUC: [0.81077303 0.87675229 0.94261763 0.85983099 0.77689156]

=======교차검증 결과=======
CV_Accuracy_mean: 0.779
CV_Precision_mean: 0.770
CV_Recall_mean: 0.791
CV_F1_스코어_mean: 0.780
CV_ROC_AUC+스코어_mean: 0.853


## LGBMClassifier

기본 모델

In [ ]:
# LGBM 모델 생성 및 학습
lgbm_model = LGBMClassifier()
lgbm_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_lgbm = lgbm_model.predict(x_test)

accuracy_lgbm = accuracy_score(y_test, y_pred_lgbm)
precision_lgbm = precision_score(y_test, y_pred_lgbm)
recall_lgbm = recall_score(y_test, y_pred_lgbm)
f1_lgbm = f1_score(y_test, y_pred_lgbm)
roc_auc_lgbm = roc_auc_score(y_test, y_pred_lgbm)

print("LightGBM 모델 평가 지표:")
print(f'Accuracy: {accuracy_lgbm:.3f}')
print(f'Precision: {precision_lgbm:.3f}')
print(f'Recall: {recall_lgbm:.3f}')
print(f'F1 스코어: {f1_lgbm:.3f}')
print(f'ROC AUC 스코어: {roc_auc_lgbm:.3f}')

result_lgbm = {
    'Accuracy' : round(accuracy_lgbm, 3),
    'Precision' : round(precision_lgbm, 3),
    'Recall' : round(recall_lgbm, 3),
    'F1' : round(f1_lgbm, 3),
    'ROC AUC' : round(roc_auc_lgbm, 3)
}
default['LGBMClassifier'] = result_lgbm
default

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 128, number of negative: 128
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003868 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 885
[LightGBM] [Info] Number of data points in the train set: 256, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

In [18]:
# 파라미터 조정
param_grid = {
        'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1],
        'n_estimators': np.arange(10, 210, 10),
        'max_depth': np.arange(1, 10, 1),
        'num_leaves': np.arange(1, 200, 1),
        'min_child_samples': np.arange(1, 10, 1)
    }

lgbm = LGBMClassifier()
random = RandomizedSearchCV(lgbm, param_grid, cv = 5, n_jobs = -1, n_iter=99)
random.fit(x_train,y_train)
random_pf = random.best_estimator_.predict(x_test)

accuracy_rdrf = accuracy_score(y_test, random_pf)
precision_rdrf = precision_score(y_test, random_pf)
recall_rdrf = recall_score(y_test, random_pf)
f1_rdrf = f1_score(y_test, random_pf)
roc_auc_rdrf = roc_auc_score(y_test, random_pf)

print("LGBM 모델 하이퍼파라미터 조정 평가 지표")
print(random.best_params_)
print(f'Accuracy: {accuracy_rdrf:.3f}')
print(f'Precision: {precision_rdrf:.3f}')
print(f'Recall: {recall_rdrf:.3f}')
print(f'F1 스코어: {f1_rdrf:.3f}')
print(f'ROC AUC 스코어: {roc_auc_rdrf:.3f}')

# 파라미터 조정 결과 저장
result0_lgbm = {
    'Accuracy' : round(accuracy_rdrf, 3),
    'Precision' : round(precision_rdrf, 3),
    'Recall' : round(recall_rdrf, 3),
    'F1' : round(f1_rdrf, 3),
    'ROC AUC' : round(roc_auc_rdrf, 3),
    'Best Parameters' : random.best_params_
}
best['LGBMClassifier'] = result0_lgbm
best

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4301, number of negative: 4301
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000965 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3385
[LightGBM] [Info] Number of data points in the train set: 8602, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

,LogisticRegression,SVC_rbf,LGBMClassifier
Accuracy,0.731,0.782,0.81
Precision,0.617,0.675,0.71
Recall,0.71,0.783,0.816
ROC AUC,0.727,0.782,0.811
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l1', 'C': ...","{'kernel': 'rbf', 'gamma': 0.0597, 'C': 3.8920...","{'num_leaves': 38, 'n_estimators': 110, 'min_c..."


LGBM 최적 파라미터 검증

In [19]:
# 모델 생성
# lgbm 최적파라미터 적용 Crossvalidation
cv_model_lgbm = LGBMClassifier(num_leaves=38, n_estimators=110, min_child_samples=7, max_depth=5, learning_rate=0.1)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_lgbm, x_train, y_train, cv=7, scoring='accuracy')
cv_precision = cross_val_score(cv_model_lgbm, x_train, y_train, cv=7, scoring='precision')
cv_recall = cross_val_score(cv_model_lgbm, x_train, y_train, cv=7, scoring='recall')
cv_f1 = cross_val_score(cv_model_lgbm, x_train, y_train, cv=7, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_lgbm, x_train, y_train, cv=7, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_lgbm = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['LGBMClassifier'] = cv_result_lgbm

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 3687, number of negative: 3686
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001305 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3384
[LightGBM] [Info] Number of data points in the train set: 7373, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500068 -> initscore=0.000271
[LightGBM] [Info] Start training from score 0.000271
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

#### 딥러닝

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# import tensorflow as tf

In [ ]:
# model = Sequential()
# model.add(Dense(256, input_dim = 12, activation = 'relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation = 'relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# history = model.fit(x_train, y_train, epochs=200, batch_size=5)

# print("\n Accuracy: %.4f" % (model.evaluate(x_test, y_test)[1]))

In [ ]:
# # 모델을 사용하여 테스트 데이터에 대한 예측 수행
# y_pred = model.predict(x_test)
# y_pred = binarize(y_pred, threshold=0.5)  # 예측값을 0.5 임계값을 기준으로 이진 분류로 변환

# # 정확도(accuracy) 계산
# accuracy = accuracy_score(y_test, y_pred)
# print(f"accuracy: {accuracy:.4f}")

# # F1 점수(f1 score) 계산
# f1 = f1_score(y_test, y_pred)
# print(f"F1-Score: {f1:.4f}")

# # 재현율(recall) 계산
# recall = recall_score(y_test, y_pred)
# print(f"recall: {recall:.4f}")

# # 정밀도(precision) 계산
# precision = precision_score(y_test, y_pred)
# print(f"precision: {precision:.4f}")

## XGboost classifier

In [22]:
# 파라미터 조정
xgboost_model = XGBClassifier()

param_grid = {
    'n_estimators': np.arange(10, 200, 10),  # 트리 개수
    'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1]  # 학습률
}

random_search = RandomizedSearchCV(estimator=xgboost_model, param_distributions=param_grid, n_iter=99,
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_search.fit(x_train, y_train)

best_model = random_search.best_estimator_

y_pred_xgboost = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_xgboost = accuracy_score(y_test, y_pred_xgboost)
precision_xgboost = precision_score(y_test, y_pred_xgboost)
recall_xgboost = recall_score(y_test, y_pred_xgboost)
f1_xgboost = f1_score(y_test, y_pred_xgboost)
roc_auc_xgboost = roc_auc_score(y_test, y_pred_xgboost)

print("Best Parameters:", random_search.best_params_)
print("XGBoost 모델 하이퍼파라미터 조정 평가 지표:")
print(f'Accuracy: {accuracy_xgboost:.3f}')
print(f'Precision: {precision_xgboost:.3f}')
print(f'Recall: {recall_xgboost:.3f}')
print(f'F1 Score: {f1_xgboost:.3f}')
print(f'ROC AUC Score: {roc_auc_xgboost:.3f}')

result0_xgboost = {
    'Accuracy' : round(accuracy_xgboost, 3),
    'Precision' : round(precision_xgboost, 3),
    'Recall' : round(recall_xgboost, 3),
    'F1' : round(f1_xgboost, 3),
    'ROC AUC' : round(roc_auc_xgboost, 3),
    'Best Parameters' : random_search.best_params_
}
best['LGBMClassifier'] = result0_xgboost
best

Fitting 5 folds for each of 95 candidates, totalling 475 fits


Best Parameters: {'n_estimators': 170, 'learning_rate': 0.1}
XGBoost 모델 하이퍼파라미터 조정 평가 지표:
Accuracy: 0.810
Precision: 0.714
Recall: 0.808
F1 Score: 0.758
ROC AUC Score: 0.810


,LogisticRegression,SVC_rbf,LGBMClassifier
Accuracy,0.731,0.782,0.81
Precision,0.617,0.675,0.714
Recall,0.71,0.783,0.808
ROC AUC,0.727,0.782,0.81
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l1', 'C': ...","{'kernel': 'rbf', 'gamma': 0.0597, 'C': 3.8920...","{'n_estimators': 170, 'learning_rate': 0.1}"


In [24]:
# 최적 파라미터 검증
xgboost_model = XGBClassifier(n_estimators=170, learning_rate=0.1)

# Cross Validation
cv_accuracy = cross_val_score(xgboost_model, x_train, y_train, cv=7, scoring='accuracy')
cv_precision = cross_val_score(xgboost_model, x_train, y_train, cv=7, scoring='precision')
cv_recall = cross_val_score(xgboost_model, x_train, y_train, cv=7, scoring='recall')
cv_f1 = cross_val_score(xgboost_model, x_train, y_train, cv=7, scoring='f1')
cv_roc_auc = cross_val_score(xgboost_model, x_train, y_train, cv=7, scoring='roc_auc')

print('=======교차검증 결과=======')
print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 평균값=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_xgboost = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['XGBClassifier'] = cv_result_xgboost

=======교차검증 결과=======
CV_Accuracy_Scores: [0.7453214  0.7558991  0.85353946 0.90480065 0.86818552 0.726607
 0.72312704]
CV_Precision_Scores: [0.73047473 0.74228395 0.83282209 0.88307692 0.8647343  0.70789866
 0.71677215]
CV_Recall_Scores: [0.77687296 0.78338762 0.88436482 0.93333333 0.87317073 0.77235772
 0.73778502]
CV_F1_Scores: [0.75295975 0.76228209 0.85781991 0.90750988 0.86893204 0.73872473
 0.72712681]
CV_ROC/AUC: [0.84258891 0.84919891 0.93240115 0.9681947  0.95035619 0.82184264
 0.81910949]

=======교차검증 평균값=======
CV_Accuracy_mean: 0.797
CV_Precision_mean: 0.783
CV_Recall_mean: 0.823
CV_F1_스코어_mean: 0.802
CV_ROC_AUC_mean: 0.883


# 결과

In [ ]:
default

In [ ]:
best

In [ ]:
validation